In [1]:
import pandas as pd
import numpy as np 
import pypsa

In [2]:
n = pypsa.Network("results/networks/10/cy1985_ty2025.nc")

       'AL00-ME00-AC', 'AL00-MK00-AC', 'AL00-RS00-AC', 'AT00 battery charger',
       'AT00 battery discharger', 'AT00 reservoir charger',
       'AT00 reservoir discharger',
       ...
       'UK00-DKW1-DC', 'UK00-FR00-DC', 'UK00-IE00-DC', 'UK00-NL00-DC',
       'UK00-NOS0-DC', 'UK00-UKNI-DC', 'UKNI battery charger',
       'UKNI battery discharger', 'UKNI-IE00-AC', 'UKNI-UK00-DC'],
      dtype='object', name='Bus', length=361) for attribute mu_alegro of Bus are not in main components dataframe buses
       'DKE1-DE00', 'DKW1-DE00', 'DKW1-NL00', 'ES00-FR00', 'ITN1-AT00',
       'ITN1-FR00', 'ITN1-SI00', 'LT00-PL00', 'NOS0-DE00', 'NOS0-NL00',
       'RS00-HR00', 'RS00-HU00', 'RS00-RO00', 'SE04-DE00', 'SE04-PL00',
       'UK00-BE00', 'UK00-FR00', 'UK00-NL00'],
      dtype='object', name='Link') for attribute external_flow of Link are not in main components dataframe links
       'BA00 reservoir store', 'BE00 PHS store', 'BE00 battery store',
       'BG00 PHS store', 'BG00 reservoir stor

In [61]:
c = n.copy()

In [62]:
load_shedding = c.generators.filter(like="load-shedding", axis=0)

In [63]:
snapshots_with_curtailment = c.snapshots[c.generators_t.p[load_shedding.index].sum(axis=1)>0]

In [64]:
c.set_snapshots(snapshots_with_curtailment)

In [65]:
other_generators = c.generators.loc[c.generators.index.symmetric_difference(load_shedding.index)]

In [66]:
market_buses = c.buses.query("carrier == 'electricity'").index

In [67]:
generation = c.generators_t.p[other_generators.index].groupby(other_generators.bus, axis=1).sum()[market_buses]

In [68]:
itc_carriers  = ["AC", "DC"]

In [69]:
storage_links = c.links.query("carrier not in @itc_carriers")

In [70]:
net_positions = c.loads_t.p_set.groupby(c.loads.bus, axis=1).sum().subtract(
    c.generators_t.p[other_generators.index].groupby(other_generators.bus, axis=1).sum(),
    fill_value=0
).add(
    c.links_t.p0[storage_links.index].groupby(storage_links.bus0, axis=1).sum(),
    fill_value=0
).add(
    c.links_t.p1.groupby(storage_links.bus1, axis=1).sum(),
    fill_value=0
)[market_buses]

In [71]:
loads = c.loads_t.p_set.copy()

In [72]:
c.mremove("Generator", other_generators.index)
c.mremove("Load", c.loads.index)
c.mremove("Link", storage_links.index)
c.mremove("Store", c.stores.index)
c.mremove("Bus", c.buses.query("carrier != 'electricity'").index)
c.madd("Load", net_positions.columns, bus = net_positions.columns, p_set = net_positions)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BG00', 'CH00', 'CY00', 'CZ00', 'DE00',
       'DEKF', 'DKE1', 'DKKF', 'DKW1', 'EE00', 'ES00', 'FI00', 'FR00', 'GR00',
       'GR03', 'HR00', 'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS', 'ITN1', 'ITS1',
       'ITSA', 'ITSI', 'LT00', 'LUG1', 'LV00', 'ME00', 'MK00', 'MT00', 'NL00',
       'NOM1', 'NON1', 'NOS0', 'PL00', 'PLE0', 'PLI0', 'PT00', 'RO00', 'RS00',
       'SE01', 'SE02', 'SE03', 'SE04', 'SI00', 'SK00', 'UK00', 'UKNI'],
      dtype='object')

In [144]:
for ls in load_shedding.index:
    c.generators_t.p_max_pu[ls] = 1

In [153]:
c.generators_t.p_max_pu[net_positions[[i[:4] for i in load_shedding.index]].values >0] = 0

In [156]:
snapshots = c.snapshots

In [157]:
m = c.optimize.create_model()

In [158]:
max_curtailment_ratio = m.add_variables(lower=0, coords=[snapshots] , name="Bus-max_curtailment_ratio")

In [159]:
loads.columns.name = "bus"

In [160]:
loads = loads.reindex(c.buses.index, axis=1, fill_value=1e-6)

In [161]:
m.add_constraints(
    max_curtailment_ratio >= m["Generator-p"].groupby(c.generators.bus).sum()/loads.stack().to_xarray(),
    name="Bus-max_curtailment_ratio_constraint"
)

Constraint `Bus-max_curtailment_ratio_constraint` (snapshot: 62, bus: 53):
--------------------------------------------------------------------------
[2010-01-15 17:00:00, AL00]: +1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] - 0.0006313 Generator-p[2010-01-15 17:00:00, AL00 load-shedding] ≥ -0.0
[2010-01-15 17:00:00, AT00]: +1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] - 8.245e-05 Generator-p[2010-01-15 17:00:00, AT00 load-shedding] ≥ -0.0
[2010-01-15 17:00:00, BA00]: +1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] - 0.0005095 Generator-p[2010-01-15 17:00:00, BA00 load-shedding] ≥ -0.0
[2010-01-15 17:00:00, BE00]: +1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] - 6.39e-05 Generator-p[2010-01-15 17:00:00, BE00 load-shedding]  ≥ -0.0
[2010-01-15 17:00:00, BG00]: +1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] - 0.0001612 Generator-p[2010-01-15 17:00:00, BG00 load-shedding] ≥ -0.0
[2010-01-15 17:00:00, CH00]: +1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] - 8.743e-05 Gene

In [162]:
m.add_objective(
    max_curtailment_ratio.sum(),
    overwrite=True
)

LinearExpression
----------------
+1 Bus-max_curtailment_ratio[2010-01-15 17:00:00] + 1 Bus-max_curtailment_ratio[2010-01-16 06:00:00] + 1 Bus-max_curtailment_ratio[2010-01-16 07:00:00] ... +1 Bus-max_curtailment_ratio[2010-11-09 21:00:00] + 1 Bus-max_curtailment_ratio[2010-11-09 22:00:00] + 1 Bus-max_curtailment_ratio[2010-11-09 23:00:00]

In [163]:
m.solve(solver_name="cplex")

INFO:linopy.model: Solve problem using Cplex solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 85.15it/s]
INFO:linopy.io: Writing time: 0.24s


Checking license ...


cpxchecklic: /lib64/libcurl.so.4: no version information available (required by cpxchecklic)



License found. [0.04 s]
Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 32 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 30 threads...
 * Starting primal Simplex on 1 thread...
Row 'c32302' infeasible, all entries at implied bounds.
Presolve time = 0.02 sec. (4.68 ticks)


Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: infeasible



('warning', 'infeasible')

In [137]:
n.generators_t.p[load_shedding.index].sum()

Generator
AL00 load-shedding        0.000000
AT00 load-shedding    14995.914453
BA00 load-shedding        0.000000
BE00 load-shedding    16262.045326
BG00 load-shedding        0.000000
CH00 load-shedding        0.000000
CY00 load-shedding       32.914023
CZ00 load-shedding        0.000000
DE00 load-shedding        0.000000
DEKF load-shedding        0.000000
DKE1 load-shedding     2287.894143
DKKF load-shedding      179.917393
DKW1 load-shedding        0.000000
EE00 load-shedding     3805.739298
ES00 load-shedding        0.000000
FI00 load-shedding     5956.583828
FR00 load-shedding    13689.039392
GR00 load-shedding        0.000000
GR03 load-shedding        0.000000
HR00 load-shedding        0.000000
HU00 load-shedding        0.000000
IE00 load-shedding    11066.889783
ITCA load-shedding        0.000000
ITCN load-shedding        0.000000
ITCS load-shedding        0.000000
ITN1 load-shedding        0.000000
ITS1 load-shedding        0.000000
ITSA load-shedding        0.000000
ITSI load-

In [135]:
c.generators_t.p.sum().round(0)

Generator
AL00 load-shedding     2835.0
AT00 load-shedding     1057.0
BA00 load-shedding        0.0
BE00 load-shedding     3781.0
BG00 load-shedding      999.0
CH00 load-shedding        0.0
CY00 load-shedding       33.0
CZ00 load-shedding        0.0
DE00 load-shedding        0.0
DEKF load-shedding        0.0
DKE1 load-shedding      368.0
DKKF load-shedding        0.0
DKW1 load-shedding      210.0
EE00 load-shedding     5770.0
ES00 load-shedding        0.0
FI00 load-shedding     2061.0
FR00 load-shedding    27580.0
GR00 load-shedding        0.0
GR03 load-shedding        0.0
HR00 load-shedding      684.0
HU00 load-shedding     1740.0
IE00 load-shedding     3377.0
ITCA load-shedding       -0.0
ITCN load-shedding        0.0
ITCS load-shedding        0.0
ITN1 load-shedding        0.0
ITS1 load-shedding        0.0
ITSA load-shedding        0.0
ITSI load-shedding        0.0
LT00 load-shedding      527.0
LUG1 load-shedding        0.0
LV00 load-shedding        0.0
ME00 load-shedding        0.0
